In [1]:
# !git clone https://github.com/fadhleryani/malti_arabi_fst.git
# # !git pull

# %pip install pynini
# %pip install pyfoma

In [2]:
# %cd malti_arabi_fst

In [1]:
import pynini as pn
import kenlm
from itertools import product


In [2]:
malti2arabi = pn.string_file('malti2arabi.map')


malti_sigma = pn.project(malti2arabi,'input')
arabi_sigma = pn.project(malti2arabi,'output') 


sigma_input = pn.union(malti_sigma ).closure()

transcriber = pn.union(malti2arabi ).closure()



dediac_cross = pn.string_file('dediac.map')

dediac = pn.cdrewrite(dediac_cross,'','',arabi_sigma.closure())

words = pn.string_file('tn-maghreb-words.txt') @ dediac


In [3]:
def get_paths(fst):
    return list(fst.paths().items())


# get_paths('merħba' @ rwrs @ transcriber)



fst = ('<BOS>ustad<EOS>' )  @ transcriber @ dediac @ words

get_paths(fst)

[('<BOS>ustad<EOS>', 'ستاد', <tropical Weight 0 at 0x10598b2f0>),
 ('<BOS>ustad<EOS>', 'استاذ', <tropical Weight 0 at 0x10598b270>)]

In [4]:
def translit_token(tok):
    fst = (f'<BOS>{tok}<EOS>')  @ transcriber @ dediac @ pn.cdrewrite(pn.cross('+ ',''),'','',arabi_sigma.closure()) @ words
    paths = [x[1] for x in get_paths(fst)]
    return paths

def translit_sentence(sentence):
    lattice = []
    for tok in sentence.split():
        options = translit_token(tok)
        lattice.append(options)

    return [' '.join(x) for x in list(product(*lattice))]

translit_sentence('min feyn')

['من فين', 'من فاين', 'مين فين', 'مين فاين']

In [5]:
fst = (f'<BOS>feyn<EOS>')  @ transcriber  @ pn.cdrewrite(pn.cross('+ ',''),'','',arabi_sigma.closure()) @ dediac
get_paths(fst)

[('<BOS>feyn<EOS>', 'فين', <tropical Weight 0 at 0x10598b7f0>),
 ('<BOS>feyn<EOS>', 'فاين', <tropical Weight 0 at 0x10598b210>)]